# Interpreting Large Language Models with Soft Entropy

There are major questions about multi-linguality from both a cognitive perspective and a typological perspective. An advantage of working with large language models is that they allow you to look at what happens inside the model on a representational level as it processes a sentence. We've prepared a number of datasets and models for you to look at, all listed in the ```1_resources.ipynb``` notebook. A description of the measures can be found in the file ```3_entropy_explanation.ipynb``` or in [this paper](https://arxiv.org/pdf/2406.02449).



In [ ]:
#We import a class for analysing BERT models and an entropy estimator
from h import BERTAnalyser, Estimator

In [ ]:
languages = ['french', 'german', 'finnish', 'romanian', 'danish', 'spanish']

In [ ]:
save_path = 'results' #path where JSON file with results should be written
model_id = 'distilbert/distilbert-base-multilingual-cased' #hugging face model id
task_repo, subtask_name = 'hcoxec/french_german_mix', None #hugging face model repo and substask names
n_batches, batch_size = 10, 256 #how many batches of data to analyse, and sentences per batch
device = 'cpu' #torch device to load model into usually cpu or cuda:0
cache_dir = 'model_cache' #path to directory where model files should be saved

In [ ]:
#these additional logs will also be written to the results file, to help
#keep track of variables later
additional_logs = {
    "model" : 'm_bert',
    "n_examples" : n_batches*batch_size,
}

In [ ]:
h_estimator = Estimator() #instantiates and Entropy estimator
analyser = BERTAnalyser( #intantiants an analyser and loads the bert model into memory
    device=device,
    h_estimator=h_estimator, 
    model_id=model_id, 
    results_file_name=save_path,
    cache_dir=cache_dir
)
analyser.get_dataset(task_repo, subtask_name) #downloads the dataset
train_loader = analyser.get_dataloader(bs=batch_size) #packages the dataset into batches


In [ ]:
h_estimator.reset() #clears the probability table from the estimator to start a new analysis
results_dict, results_df  = analyser.online_estimation( #runs online entropy estimation for the number of batches specified
    train_loader, 
    max_batches=n_batches, 
    additional_logs= additional_logs,
    label_types=['language', 'token','bigram','trigram']

)
results_df

In [ ]:
analyser.plot_residual(results_df) #builds a pie chart of the residual

In [ ]:
analyser.plot(results_df, measure='disentanglement') #generates a bar plot of any measure

In [ ]:
import pandas as pd

#loads all saved results into a dataframe for analysis
df = pd.read_json('results.json', lines=True)
